In [2]:
pip install scikeras[tensorflow]

In [3]:
#not show feature warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd
import tensorflow as tf
import numpy as np


from scikeras.wrappers import KerasClassifier

from sklearn.metrics import accuracy_score, classification_report, recall_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline

# from wittgenstein import RIPPER
# from wittgenstein.interpret import interpret_model, score_fidelity


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
src = "/content/drive/MyDrive/Deep Learning/40 disease/dataset_modified.csv"
data = pd.read_csv(src)
le = LabelEncoder()
for col in data.select_dtypes(include='object'):
    data[col] = le.fit_transform(data[col])
# View updated dataframe
X = data.drop(['Disease'], axis=1)
y = data["Disease"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

x_train = x_train.to_numpy(dtype='float32')
x_test = x_test.to_numpy(dtype='float32')

# Convert target data to numpy arrays with int64 data type
y_train = y_train.to_numpy(dtype='int64')
y_test = y_test.to_numpy(dtype='int64')

In [6]:
y.nunique()

41

In [7]:
num_classes = y.nunique()
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=32, input_shape=(x_train.shape[1], 1), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.LSTM(units=16, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#rnn_model = KerasClassifier(build_fn=model, epochs=100, batch_size=32, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))


Epoch 1/100
116/116 [==============================] - 20s 136ms/step - loss: 3.7146 - accuracy: 0.0241 - val_loss: 3.7146 - val_accuracy: 0.0187
Epoch 2/100
116/116 [==============================] - 16s 139ms/step - loss: 3.7136 - accuracy: 0.0225 - val_loss: 3.7123 - val_accuracy: 0.0171
Epoch 3/100
116/116 [==============================] - 14s 124ms/step - loss: 3.5727 - accuracy: 0.0621 - val_loss: 3.1595 - val_accuracy: 0.1236
Epoch 4/100
116/116 [==============================] - 15s 128ms/step - loss: 2.8174 - accuracy: 0.1894 - val_loss: 2.4591 - val_accuracy: 0.3000
Epoch 5/100
116/116 [==============================] - 16s 142ms/step - loss: 2.3336 - accuracy: 0.3520 - val_loss: 1.9939 - val_accuracy: 0.6293
Epoch 6/100
116/116 [==============================] - 15s 126ms/step - loss: 1.9938 - accuracy: 0.5079 - val_loss: 1.7689 - val_accuracy: 0.6902
Epoch 7/100
116/116 [==============================] - 15s 128ms/step - loss: 1.7457 - accuracy: 0.5886 - val_loss: 1.6018 -

In [8]:
rnn_model = KerasClassifier(build_fn=model, epochs=2, batch_size=32, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [9]:
# adaboost_model = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='use_encoded_value'), AdaBoostClassifier(learning_rate=.1))
adaboost_model = AdaBoostClassifier()
gradientboost_model = GradientBoostingClassifier(criterion='squared_error')

In [10]:
ensemble_clf = StackingClassifier(estimators=[('adaboost', adaboost_model), ('rnn', rnn_model), ('gradientboost', gradientboost_model)], final_estimator=LogisticRegression())
ensemble_clf.fit(x_train, y_train)

Epoch 1/2
116/116 [==============================] - 18s 133ms/step - loss: 0.2695 - accuracy: 0.9165
Epoch 2/2
116/116 [==============================] - 14s 120ms/step - loss: 0.2932 - accuracy: 0.9092
Epoch 1/2
93/93 [==============================] - 14s 106ms/step - loss: 0.2901 - accuracy: 0.9129
Epoch 2/2
24/24 [==============================] - 2s 27ms/step
Epoch 1/2
93/93 [==============================] - 14s 123ms/step - loss: 0.2581 - accuracy: 0.9238
Epoch 2/2
24/24 [==============================] - 1s 27ms/step


Epoch 1/2
93/93 [==============================] - 14s 122ms/step - loss: 0.3378 - accuracy: 0.8940
Epoch 2/2
24/24 [==============================] - 2s 26ms/step


Epoch 1/2
93/93 [==============================] - 14s 124ms/step - loss: 0.2804 - accuracy: 0.9133
Epoch 2/2
24/24 [==============================] - 1s 26ms/step


Epoch 1/2
93/93 [==============================] - 14s 126ms/step - loss: 0.3286 - accuracy: 0.8879
Epoch 2/2
24/24 [==============================] - 1s 27ms/step


StackingClassifier(estimators=[('adaboost', AdaBoostClassifier()),
                               ('rnn',
                                KerasClassifier(batch_size=32, build_fn=<keras.src.engine.sequential.Sequential object at 0x789199b3a230>, epochs=2, loss=<keras.src.losses.SparseCategoricalCrossentropy object at 0x789199e53550>)),
                               ('gradientboost',
                                GradientBoostingClassifier(criterion='squared_error'))],
                   final_estimator=LogisticRegression())

In [11]:
y_pred = ensemble_clf.predict(x_test)
print("Classification report: ")
print(classification_report(y_test, y_pred))

print("Accuracy: ")
print(accuracy_score(y_test, y_pred))

print("Recall: ")
print(recall_score(y_test, y_pred, average=None))

39/39 [==============================] - 2s 28ms/step
Classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        29
           5       1.00      1.00      1.00        30
           6       1.00      1.00      1.00        38
           7       1.00      1.00      1.00        29
           8       1.00      1.00      1.00        30
           9       1.00      1.00      1.00        21
          10       1.00      1.00      1.00        29
          11       1.00      1.00      1.00        29
          12       1.00      1.00      1.00        29
          13       1.00      1.00      1.00        37
          14       1.00      1.00      1.00        30
          15       1.00      1.00      1.00        23
   